In [10]:
key = ''


In [11]:
import os
import openai
from openai import OpenAI

In [12]:
openai.api_key=key
client=OpenAI(api_key=key)

In [4]:
def get_cvss_metrics(description):
    prompt = f"""
    Analyze the following CVE description and calculate the CVSS v3.1 Base Score. Determine the values for each base metric: AV, AC, PR, UI, S, C, I, and A. Summarize each metric's value and provide the final CVSS v3.1 vector string.
    
    CVE Description: {description}
    
    Valid options for each metric are as follows:
    - Attack Vector (AV): Network (N), Adjacent (A), Local (L), Physical (P)
    - Attack Complexity (AC): Low (L), High (H)
    - Privileges Required (PR): None (N), Low (L), High (H)
    - User Interaction (UI): None (N), Required (R)
    - Scope (S): Unchanged (U), Changed (C)
    - Confidentiality (C): None (N), Low (L), High (H)
    - Integrity (I): None (N), Low (L), High (H)
    - Availability (A): None (N), Low (L), High (H)
    
    Ensure the final line of your response contains only the CVSS v3 Vector String in the following format:
    Example format: CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H
    """
    

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an expert in cybersecurity and CWE mappings."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=300
        )

        if response.choices and hasattr(response.choices[0].message, 'content'):
            content = response.choices[0].message.content.strip()
            result = content.split('\n')[-1]  # Get the last line
            print(f"Generated CVSS: {result}")  # Debug print
            return result
        else:
            print("Error: Invalid response from OpenAI API")
            return None

    except openai.RateLimitError as e:
        print(f"Rate limit error: {e}")
        time.sleep(2 ** random.randint(1, 5))  # Exponential backoff
        return generate_gt(cve_description)  # Retry

    except Exception as e:
        print(f"Error: {e}")
        return None

In [5]:
import pandas as pd 

df=pd.read_csv('myVSPData.tsv',sep='\t').sample(n=100)
df.head()

,CVE_ID,Description,CWE,CVSS_Vector
576,CVE-2024-1008,A vulnerability was found in SourceCodester Em...,CWE-434,CVSS:3.1/AV:N/AC:L/PR:H/UI:N/S:U/C:H/I:H/A:H
336,CVE-2024-0594,The Awesome Support – WordPress HelpDesk & Sup...,CWE-89,CVSS:3.1/AV:N/AC:L/PR:L/UI:N/S:U/C:H/I:H/A:H
569,CVE-2024-1001,A vulnerability classified as critical has bee...,CWE-121,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H
897,CVE-2024-20653,Microsoft Common Log File System Elevation of ...,NVD-CWE-noinfo,CVSS:3.1/AV:L/AC:L/PR:L/UI:N/S:U/C:H/I:H/A:H
770,CVE-2024-1879,A Cross-Site Request Forgery (CSRF) vulnerabil...,CWE-352,CVSS:3.1/AV:N/AC:L/PR:N/UI:R/S:U/C:H/I:H/A:H


In [14]:
# List to store CVSS results
cvss_results = []

# Iterate over the DataFrame and get CVSS metrics from GPT-3
for index, row in df.iterrows():
    description = row["Description"]
    metrics = get_cvss_metrics(description)
    cvss_results.append(metrics)

# Add CVSS results to the DataFrame
df['CVSS'] = cvss_results

# Save the results to a new TSV file
df.to_csv('resMyVSP.tsv', sep='\t', index=False)

# Display the DataFrame with CVSS results
print(df[[ 'Description', 'CVSS']])

Generated CVSS: CVSS:3.1/AV:N/AC:H/PR:N/UI:N/S:U/C:H/I:H/A:H
Generated CVSS: CVSS:3.1/AV:N/AC:L/PR:L/UI:N/S:U/C:H/I:H/A:N
Generated CVSS: Therefore, the CVSS v3.1 vector string would be: CVSS:3.1/AV:N/AC:H/PR:N/UI:N/S:U/C:H/I:H/A:H
Generated CVSS: - Base Score = 7.8 (CVSS:
Generated CVSS:    Explanation: The vulnerability allows attackers to execute arbitrary
Generated CVSS: Final CVSS v3.1 Vector String: CVSS:3.1/AV:L/AC:H/PR:L/UI:N/S:U/C:L/I:H/A:N
Generated CVSS: CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H
Generated CVSS: CVSS:3.1/AV:N/AC:L/PR:L/UI:N/S:U/C:H/I:L/A:N
Generated CVSS: CVSS:3.1/AV:L/AC:L/PR:N/UI:
Generated CVSS: CVSS:3.1/AV:N/AC:L/PR:L/UI:N/S:U/C:H/I:H/A:H
Generated CVSS: CVSS:3.1/AV:N/AC:L/PR:H/UI:N/S:U/C:H/I:H/A:H
Generated CVSS: CVSS v3.1 Vector String: CVSS:3.1/AV:N/AC:L/PR:L/UI:N/S:U/C:H/I:H/A:H
Generated CVSS: CVSS:3.1/AV:N/AC:H/PR:N/UI:N/S:U/C:N/I:L/A:N
Generated CVSS: 8. Availability (A): Availability impact is High (H) since the exploit may lead to the componen

In [15]:
from cvss import CVSS3
def get_cvss_score(cvss_vector):
    try:
        cvss = CVSS3(cvss_vector)
        return cvss.base_score  # Return the CVSS base score
    except Exception as e:
        print(f"Error calculating CVSS score for vector {cvss_vector}: {e}")
        raise

In [19]:
def compute_vsp_mad(fname, col):
    expected_prefix = 'CVSS:3.1/'  # Adjust prefix according to your CVSS version
    df = pd.read_csv(fname, sep='\t')
    error = 0
    total = 0
    
    for idx, row in df.iterrows():
        pred = row[col].upper()
        gt = row['CVSS_Vector'].upper()
        
        # Ensure predicted vectors have the correct prefix
        if not pred.startswith(expected_prefix):
            pred = expected_prefix + pred.lstrip("CVSS:3.0/").lstrip("CVSS:3.1/")
        
        try:
            # Calculate scores for predicted and ground truth vectors
            pred_score = get_cvss_score(pred)
            gt_score = get_cvss_score(gt)
            error += abs(pred_score - gt_score)
        except Exception as e:
            print(f'Invalid response at row {idx + 1}')
            print(e)
            continue
            
        total += 1

    return error / total if total > 0 else float('nan')


In [20]:
print('Mean Absolute Deviation:', compute_vsp_mad('resMyVSP.tsv','CVSS'))

Error calculating CVSS score for vector CVSS:3.1/THEREFORE, THE CVSS V3.1 VECTOR STRING WOULD BE: CVSS:3.1/AV:N/AC:H/PR:N/UI:N/S:U/C:H/I:H/A:H: Malformed CVSS3 field "THEREFORE, THE CVSS V3.1 VECTOR STRING WOULD BE: CVSS:3.1"
Invalid response at row 3
Malformed CVSS3 field "THEREFORE, THE CVSS V3.1 VECTOR STRING WOULD BE: CVSS:3.1"
Error calculating CVSS score for vector CVSS:3.1/- BASE SCORE = 7.8 (CVSS:: Unknown metric "- BASE SCORE = 7.8 (CVSS" in field "- BASE SCORE = 7.8 (CVSS:"
Invalid response at row 4
Unknown metric "- BASE SCORE = 7.8 (CVSS" in field "- BASE SCORE = 7.8 (CVSS:"
Error calculating CVSS score for vector CVSS:3.1/   EXPLANATION: THE VULNERABILITY ALLOWS ATTACKERS TO EXECUTE ARBITRARY: Unknown metric "   EXPLANATION" in field "   EXPLANATION: THE VULNERABILITY ALLOWS ATTACKERS TO EXECUTE ARBITRARY"
Invalid response at row 5
Unknown metric "   EXPLANATION" in field "   EXPLANATION: THE VULNERABILITY ALLOWS ATTACKERS TO EXECUTE ARBITRARY"
Error calculating CVSS score